**<H1>ЛЕКСИЧНИЙ АНАЛІЗ</H1>**


# **Підготовка блокноту до роботи**

In [5]:
from typing import Tuple, Sequence, Any
from typing_extensions import Self

# **Клас регулярних виразів**

In [6]:
class ReX(tuple):

    EMT = 0  # empty expression
    NIL = 1  # nil expression
    CHR = 2  # one letter expression
    AST = 3  # Kleene closure
    CAT = 4  # concatanation
    ALT = 5  # alternation

    def __new__(cls, kind: int, *args: Sequence[Any]) -> Self:
        if type(kind) != int:
            raise ValueError("ReX() error! Bad type of expression kind")
        if kind == ReX.EMT or kind == ReX.NIL:
            if args:
                raise ValueError("ReX() error! Unexpected argument(s)")
            return super().__new__(cls, (kind, ()))
        elif kind == ReX.CHR:
            if len(args) != 1:
                raise ValueError("ReX() error! Invalid number of arguments")
            if type(args[0]) != str and len(args[0]) != 1:
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        elif kind == ReX.AST:
            if len(args) != 1:
                raise ValueError("ReX() error! Invalid number of arguments")
            if type(args[0]) != ReX:
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        elif kind == ReX.CAT or kind == ReX.ALT:
            if len(args) != 2:
                raise ValueError("ReX() error! Invalid number of arguments")
            if not all(type(arg) == ReX for arg in args):
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        else:
            raise ValueError("ReX() error! Bad kind of expression")

    @property
    def kind(self):
        return self[0]

    def __eq__(self, another: Self) -> bool:
        if type(another) != ReX:
            raise ValueError("invalid comparison")
        return super().__eq__(self, another)

    def __ne__(self, another):
        return not (self == another)

    def __str__(self):
        if self.kind == ReX.EMT:
            return "∅"
        if self.kind == ReX.NIL:
            return "ϵ"
        if self.kind == ReX.CHR:
            return self[1][0]
        if self.kind == ReX.AST:
            return f"{self[1][0]}*"
        if self.kind == ReX.CAT:
            return f"({self[1][0]} {self[1][1]})"
        # self.kind == ReX.ALT
        return f"({self[1][0]} | {self[1][1]})"

    @classmethod
    def Empty(cls) -> Self:
        return ReX(ReX.EMT)

    @classmethod
    def Nil(cls) -> Self:
        return ReX(ReX.NIL)

    @classmethod
    def Char(cls, ch: str) -> Self:
        return ReX(ReX.CHR, ch)

    @classmethod
    def Ast(cls, rex: Self) -> Self:
        return ReX(ReX.AST, rex)

    @classmethod
    def Cat(cls, rex1: Self, rex2: Self) -> Self:
        return ReX(ReX.CAT, rex1, rex2)

    @classmethod
    def Alt(cls, rex1: Self, rex2: Self) -> Self:
        return ReX(ReX.ALT, rex1, rex2)

In [7]:
empty = ReX.Empty()
print(f"empty = {empty}")
nil = ReX.Nil()
print(f"nil   = {nil}")
a, b = ReX.Char('a'), ReX.Char('b')
print(f"a  = '{a}', b = '{b}'")
ast = ReX.Ast(a)
print(f"ast   = {ast}")
cat = ReX.Cat(a, b)
print(f"cat   = {cat}")
alt = ReX.Alt(a, b)
print(f"alt   = {alt}")

empty = ∅
nil   = ϵ
a  = 'a', b = 'b'
ast   = a*
cat   = (a b)
alt   = (a | b)


# **Функція** `simplify()`

Функція `simplify(expr: ReX) -> ReX`
забезпечує рекурсивне спрощення регулярних виразів за правилами

- $(\emptyset\mid e)\longrightarrow e$
- $(e\mid\emptyset)\longrightarrow e$
- $(\emptyset\ e)\longrightarrow\emptyset$
- $(e\ \emptyset)\longrightarrow\emptyset$
- $(\epsilon\ e)\longrightarrow e$
- $(e\ \epsilon)\longrightarrow e$
- $(\emptyset)^\ast\longrightarrow\epsilon$
- $(\epsilon)^\ast\longrightarrow\epsilon$

In [8]:
def simplify(rex: ReX) -> ReX:
    kind = rex.kind
    if kind == ReX.ALT:
        left, right = rex[1]
        if left == ReX.Empty():
            return simplify(right)
        if right == ReX.Empty():
            return simplify(left)
    elif kind == ReX.CAT:
        left, right = rex[1]
        if left == ReX.Empty() or right == ReX.Empty():
            return ReX.Empty()
        if left == ReX.Nil():
            return simplify(right)
        if right == ReX.Nil():
            return simplify(left)
    elif kind == ReX.AST:
        sub_expr = rex[1][0]
        if sub_expr == ReX.Empty() or sub_expr == ReX.Nil():
            return ReX.Nil()
    return rex


# **Функція** `check_nil()`

Функція `does_contain_nil(rex: ReX) -> bool`
перевіряє чи належить порожнє слово регулярній мові, що визначається регулярним виразом `ReX`.

In [9]:
def does_contain_nil(rex: ReX) -> bool:
    kind = rex.kind
    if kind == ReX.NIL:
        return True
    elif kind == ReX.EMT:
        return False
    elif kind == ReX.CHR:
        return False
    elif kind == ReX.AST:
        return True
    elif kind == ReX.CAT:
        left, right = rex[1]
        return does_contain_nil(left) and does_contain_nil(right)
    elif kind == ReX.ALT:
        left, right = rex[1]
        return does_contain_nil(left) or does_contain_nil(right)
    return False


# **Функція** `Brzozowski()`

Функція `Brzozowski(rex: ReX, ch: str) -> ReX`
обчислює похідну Бжозовськи за символом `ch` регулярного виразу `rex`.

In [10]:
def Brzozowski(rex: ReX, ch: str) -> ReX:
    kind = rex.kind
    if kind == ReX.EMT:
        return ReX.Empty()
    elif kind == ReX.NIL:
        return ReX.Empty()
    elif kind == ReX.CHR:
        return ReX.Nil() if rex[1][0] == ch else ReX.Empty()
    elif kind == ReX.ALT:
        left, right = rex[1]
        return simplify(ReX.Alt(Brzozowski(left, ch), Brzozowski(right, ch)))
    elif kind == ReX.CAT:
        left, right = rex[1]
        if does_contain_nil(left):
            return simplify(ReX.Alt(ReX.Cat(Brzozowski(left, ch), right), Brzozowski(right, ch)))
        else:
            return simplify(ReX.Cat(Brzozowski(left, ch), right))
    elif kind == ReX.AST:
        sub_expr = rex[1][0]
        return simplify(ReX.Cat(Brzozowski(sub_expr, ch), rex))


In [14]:
class ReX(tuple):
    EMT = 0  # empty expression
    NIL = 1  # nil expression
    CHR = 2  # one letter expression
    AST = 3  # Kleene closure
    CAT = 4  # concatenation
    ALT = 5  # alternation

    def __new__(cls, kind: int, *args) -> "ReX":
        if not isinstance(kind, int):
            raise ValueError("ReX() error! Bad type of expression kind")

        if kind == ReX.EMT or kind == ReX.NIL:
            if args:
                raise ValueError("ReX() error! Unexpected argument(s)")
            return super().__new__(cls, (kind,))

        elif kind == ReX.CHR:
            if len(args) != 1 or not isinstance(args[0], str) or len(args[0]) != 1:
                raise ValueError("ReX() error! Invalid character argument")
            return super().__new__(cls, (kind, args[0]))

        elif kind == ReX.AST:
            if len(args) != 1 or not isinstance(args[0], ReX):
                raise ValueError("ReX() error! Invalid argument for AST")
            return super().__new__(cls, (kind, args[0]))

        elif kind in (ReX.CAT, ReX.ALT):
            if len(args) != 2 or not all(isinstance(arg, ReX) for arg in args):
                raise ValueError("ReX() error! Invalid arguments for CAT/ALT")
            return super().__new__(cls, (kind, args[0], args[1]))

        else:
            raise ValueError("ReX() error! Invalid kind of expression")

    @property
    def kind(self):
        return self[0]

    def __str__(self):
        if self.kind == ReX.EMT:
            return "∅"
        elif self.kind == ReX.NIL:
            return "ϵ"
        elif self.kind == ReX.CHR:
            return self[1]
        elif self.kind == ReX.AST:
            return f"{self[1]}*"
        elif self.kind == ReX.CAT:
            return f"({self[1]} {self[2]})"
        elif self.kind == ReX.ALT:
            return f"({self[1]} | {self[2]})"

    # Additional methods (Empty, Nil, Char, Ast, Cat, Alt) remain the same.

# Testing
empty = ReX(ReX.EMT)
nil = ReX(ReX.NIL)
a = ReX(ReX.CHR, 'a')
b = ReX(ReX.CHR, 'b')
cat = ReX(ReX.CAT, a, b)
alt = ReX(ReX.ALT, a, b)

print(f"empty = {empty}")
print(f"nil = {nil}")
print(f"a = {a}")
print(f"b = {b}")
print(f"cat = {cat}")
print(f"alt = {alt}")



empty = ∅
nil = ϵ
a = a
b = b
cat = (a b)
alt = (a | b)
